In [0]:

from __future__ import print_function, division
import numpy as np
import os
import cv2
from PIL import Image
import random
from functools import partial
from utils import *

import tensorflow as tf
from keras.models import Model, Sequential, load_model
from keras.layers.merge import _Merge
from keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D, Conv2D, BatchNormalization, LeakyReLU, ReLU, UpSampling2D
from keras.layers import Reshape, Dropout, Concatenate, Lambda, Multiply, Add, Flatten, Dense
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [0]:
def ResidualBlock(inp, dim_out):
  """Residual Block with instance normalization."""
  x = ZeroPadding2D(padding = 1)(inp)
  x = Conv2D(filters = dim_out, kernel_size=3, strides=1, padding='valid', use_bias = False)(x)
  x = InstanceNormalization(axis = -1)(x)
  x = ReLU()(x)
  x = ZeroPadding2D(padding = 1)(x)
  x = Conv2D(filters = dim_out, kernel_size=3, strides=1, padding='valid', use_bias = False)(x)
  x = InstanceNormalization(axis = -1)(x)
  return Add()([inp, x])

In [0]:
#generator is build based on CycleGAN
def generator_model():
  # Input tensors
  image_size=128
  domain_dim=5 # this is the domain size for our experiment
  g_conv_dim=64 #'number of conv filters in the first layer of G'
  curr_dim = g_conv_dim # initial number of convolution filter which is 64
  repeat_num=6
  #creating two tensors of domain aand input image
  inp_domain = Input(shape = (domain_dim, ))
  #print(inp_domain.shape)#(?, 5)
  inp_img = Input(shape = (image_size, image_size, 3))
  #print(inp_img.shape) (?, 128, 128, 3)

  # Replicate spatially and concatenate domain information using lambda function
  c = Lambda(lambda x: K.repeat(x, image_size**2))(inp_domain)
  #(?,16384,5)
  c = Reshape((image_size, image_size, domain_dim))(c)
  #(?, 128, 128, 5)
  x = Concatenate()([inp_img, c])
  #(?, 128, 128, 8)


  # First Conv2D We are using instance normalisation as per stated in paper
  x = Conv2D(filters = g_conv_dim, kernel_size = 7, strides = 1, padding = 'same', use_bias = False)(x)
  x = InstanceNormalization(axis = -1)(x)
  x = ReLU()(x)

  # Down-sampling layers twice as per the architecture
  
  for i in range(2):
    x = ZeroPadding2D(padding = 1)(x)
    x = Conv2D(filters = curr_dim*2, kernel_size = 4, strides = 2, padding = 'valid', use_bias = False)(x)
    x = InstanceNormalization(axis = -1)(x)
    x = ReLU()(x)
    curr_dim = curr_dim * 2
  # We are using Bottleneck layers to reduce the dimensionality of input
  for i in range(repeat_num):
    x = ResidualBlock(x, curr_dim) 
  # Finally Up-sampling layers
  for i in range(2):
    x = UpSampling2D(size = 2)(x)       
    x = Conv2D(filters = curr_dim // 2, kernel_size = 4, strides = 1, padding = 'same', use_bias = False)(x)
    x = InstanceNormalization(axis = -1)(x)
    x = ReLU()(x)        
    curr_dim = curr_dim // 2
    
  # Last Conv2D
  x = ZeroPadding2D(padding = 3)(x)
  out = Conv2D(filters = 3, kernel_size = 7, strides = 1, padding = 'valid', activation = 'tanh', use_bias = False)(x)
  # Model.summary()
  return Model(inputs = [inp_img, inp_domain], outputs = out)        
  

  

In [0]:
#discriminator is build based on PatchGAN
def discriminator_model():
  image_size=128
  d_conv_dim=64 #number of conv filters in the first layer of D
  domain_dim = 5 # this is the domain size for our experiment
  repeat_num=6 #number of strided conv layers in D given in architecture
  inp_img = Input(shape = (image_size, image_size, 3))
  x = ZeroPadding2D(padding = 1)(inp_img)
  x = Conv2D(filters = d_conv_dim, kernel_size = 4, strides = 2, padding = 'valid', use_bias = False)(x)
  x = LeakyReLU(0.01)(x)

  curr_dim = d_conv_dim
  #We are downsampling this 6 times
  for i in range(1, repeat_num):
    x = ZeroPadding2D(padding = 1)(x)
    x = Conv2D(filters = curr_dim*2, kernel_size = 4, strides = 2, padding = 'valid')(x)
    x = LeakyReLU(0.01)(x)
    curr_dim = curr_dim * 2

  #discriminator outputs whether the image is real or fake and corresponding class label
  out_src = ZeroPadding2D(padding = 1)(x)
  out_src = Conv2D(filters = 1, kernel_size = 3, strides = 1, padding = 'valid', use_bias = False)(out_src)
  kernel_size = int(image_size / np.power(2, repeat_num))

  out_cls = Conv2D(filters = domain_dim, kernel_size = kernel_size, strides = 1, padding = 'valid', use_bias = False)(x)
  out_cls = Reshape((domain_dim, ))(out_cls)

  return Model(inp_img, [out_src, out_cls])


In [0]:
#Domain classification loss
def classification_loss( Y_true, Y_pred) :
  return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_true, logits=Y_pred))

In [0]:
#wasserstein loss
def wasserstein_loss(Y_true, Y_pred):
  return K.mean(Y_true*Y_pred)

In [0]:
#reconstruction loss
def reconstruction_loss(Y_true, Y_pred):
  return K.mean(K.abs(Y_true - Y_pred))

In [0]:
def gradient_penalty_loss( y_true, y_pred, averaged_samples):
  """
  Computes gradient penalty based on prediction and weighted real / fake samples  
  This is used to stabilise training and generate higher quality images, we replace generic adversarial loss with gradient penalty
  """
  gradients = K.gradients(y_pred, averaged_samples)[0]
  # compute the euclidean norm by squaring ...
  gradients_sqr = K.square(gradients)
  # summing over the rows
  gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
  #taking the sqrt
  gradient_l2_norm = K.sqrt(gradients_sqr_sum)
  #computing the gradient penalty
  gradient_penalty = K.square(1 - gradient_l2_norm)
  # return the mean as loss over all the batch samples
  return K.mean(gradient_penalty)

In [0]:
"""
#Just to print the model summary
print("Generator model summary")
Generator=generator_model()
Generator.summary()

print("Discriminator model summary")
Discriminator= discriminator_model()
Discriminator.summary()
"""

'\nprint("Generator model summary")\nGenerator=generator_model()\nGenerator.summary()\n\nprint("Discriminator model summary")\nDiscriminator= discriminator_model()\nDiscriminator.summary()\n'

In [0]:
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def define_batch_size(self, bs):
        self.bs = bs
    def _merge_function(self, inputs):
        alpha = K.random_uniform((self.bs, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

In [0]:
def train_and_test_model():
    G = generator_model()
    D = discriminator_model()
    image_size=128
    batch_size=4 # we are keeping the batch_size as 4
    domain_dim=5 # domain size
    d_lr=0.0001 #learning rate of discriminator
    g_lr=0.0001 #learning rate of generator
    #hyperparameters for experiment
    beta1 = 0.5 
    beta2 = 0.999 
    lambda_cls = 1
    lambda_rec = 10
    lambda_gp = 10 
    sample_step = 1000
    num_iter_done = 160000 # since GPU used to heat up, I saved models and continued fom next iterations 
    # we don't want to update the weights of Generator first
    G.trainable=False
    data_dir ='celeba'
    selected_attributes =['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young']
    mode='train'

    #compute the output with real weights
    x_real = Input(shape = (image_size, image_size, 3))
    out_src_real, out_cls_real = D(x_real)

    # Compute output with fake images.
    label_trg = Input(shape = (domain_dim, ))
    x_fake = G([x_real, label_trg])
    out_src_fake, out_cls_fake = D(x_fake)

    # Compute output for gradient penalty.
    rd_avg = RandomWeightedAverage()
    rd_avg.define_batch_size(batch_size)
    x_hat = rd_avg([x_real, x_fake])
    out_src, _ = D(x_hat)


    # Use Python partial to provide loss function with additional 'averaged_samples' argument
    partial_gp_loss = partial(gradient_penalty_loss, averaged_samples=x_hat)
    partial_gp_loss.__name__ = 'gradient_penalty'


    # Define training model D
    train_D = Model([x_real, label_trg], [out_src_real, out_cls_real, out_src_fake, out_src])

    train_D.load_weights('previous_model/train_D_weights.hdf5')
    
    # Setup loss for train_D
    train_D.compile(loss = [wasserstein_loss, classification_loss, wasserstein_loss, partial_gp_loss], 
                             optimizer=Adam(lr = d_lr, beta_1 = beta1, beta_2 = beta2), loss_weights = [1,lambda_cls, 1, lambda_gp])
    #keeping the discriminator trainable as false    
    G.trainable = True
    D.trainable = False

    # All inputs
    real_x = Input(shape = (image_size, image_size, 3))
    org_label = Input(shape = (domain_dim, ))
    trg_label = Input(shape = (domain_dim, ))

    # Compute output of fake image
    fake_x = G([real_x, trg_label])
    fake_out_src, fake_out_cls = D(fake_x)

    # Target-to-original domain.
    x_reconst = G([fake_x, org_label])

    # Define traning model G
    train_G = Model([real_x, org_label, trg_label], [fake_out_src, fake_out_cls, x_reconst])
    
    #loading saved weights
    train_G.load_weights('previous_model/train_D_weights.hdf5')
    
    #Setup loss for train_G
    train_G.compile(loss = [wasserstein_loss, classification_loss, reconstruction_loss], 
                             optimizer=Adam(lr = g_lr, beta_1 = beta1, beta_2 = beta2), loss_weights = [1, lambda_cls, lambda_rec])

    Image_data_class = ImageData(data_dir, selected_attributes)
    Image_data_class.preprocess()


    #begining the training data set
    data_iter= get_loader(Image_data_class.train_dataset,Image_data_class.train_dataset_label,Image_data_class.train_dataset_fix_label,
                          image_size, batch_size, mode)

    # Training
    valid = -np.ones((batch_size, 2, 2, 1))
    fake =  np.ones((batch_size, 2, 2, 1))
    dummy = np.zeros((batch_size, 2, 2, 1)) # Dummy gt for gradient penalty
    log_step=10 # after every 10 steps I am printing log
    num_iters=200000 #total number of iterations
    num_iters_decay=100000 
    n_critic=5 #updating generator once evry 5 time training the discriminator
    model_save_step = 10000 #number of steps after which we will save the model
    model_save_dir='models'
    result_dir= 'results'
    G.load_weights('previous_model/G_weights.hdf5')
    D.load_weights('previous_model/D_weights.hdf5')
    for epoch in range(num_iter_done, num_iters):
      imgs, orig_labels, target_labels, fix_labels, _ = next(data_iter)

      # Setting learning rate (linear decay)
      if epoch > (num_iters - num_iters_decay):
          K.set_value(train_D.optimizer.lr, d_lr*(num_iters - epoch)/(num_iters - num_iters_decay))
          K.set_value(train_G.optimizer.lr, g_lr*(num_iters - epoch)/(num_iters - num_iters_decay))

      # Training Discriminators batchwise       
      D_loss = train_D.train_on_batch(x = [imgs, target_labels], y = [valid, orig_labels, fake, dummy])

      # Training Generators batchwise
      if (epoch + 1) % n_critic == 0:
          G_loss = train_G.train_on_batch(x = [imgs, orig_labels, target_labels], y = [valid, target_labels, imgs])
      
      #printing loss functions after every 10 steps
      if (epoch + 1) % log_step == 0:
          print(f"Iteration: [{epoch + 1}/{num_iters}]")
          print(f"\tD/loss_real = [{D_loss[1]:.4f}], D/loss_fake = [{D_loss[3]:.4f}], D/loss_cls =  [{D_loss[2]:.4f}], D/loss_gp = [{D_loss[4]:.4f}]")
          print(f"\tG/loss_fake = [{G_loss[1]:.4f}], G/loss_rec = [{G_loss[3]:.4f}], G/loss_cls = [{G_loss[2]:.4f}]") 

      # saving the model after every 10,0000 steps
      if (epoch + 1) % model_save_step == 0:  
          G.save_weights(os.path.join(model_save_dir, 'G_weights.hdf5'))
          D.save_weights(os.path.join(model_save_dir, 'D_weights.hdf5'))
          train_D.save_weights(os.path.join(model_save_dir, 'train_D_weights.hdf5'))
          train_G.save_weights(os.path.join(model_save_dir, 'train_G_weights.hdf5'))  


    # Once we are done wih the training of the model, we perform testing the generated model, so for every test image in celebA dataset we generate the images corresponding to 5 domains
    mode='test'
    G_weights_dir = os.path.join(model_save_dir, 'G_weights.hdf5') 
    if not os.path.isfile(G_weights_dir):
      print("Don't find weight's generator model")
    else:
      G.load_weights(G_weights_dir)
    data_iter = get_loader(Image_data_class.test_dataset, Image_data_class.test_dataset_label, Image_data_class.test_dataset_fix_label, 
                           image_size, batch_size, mode)
    n_batches = int(sample_step / batch_size)
    # we are getting n_batches images from test dataset, getting the generator to generate the images and saving it
    for i in range(n_batches):
        imgs, orig_labels, target_labels, fix_labels, names = next(data_iter)
        #print("image length ",len(imgs))
        for j in range(batch_size):
            #print("fixed_labels", fix_labels[j])
            preds = G.predict([np.repeat(np.expand_dims(imgs[j], axis = 0), len(selected_attributes), axis = 0), fix_labels[j]])
            for k in range(len(selected_attributes)):  
                Image.fromarray((preds[k]*127.5 + 127.5).astype(np.uint8)).save(os.path.join(result_dir, names[j].split(os.path.sep)[-1].split('.')[0] + f'_{k + 1}.png')) 


In [0]:
train_and_test_model()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

 Finished preprocessing the CelebA dataset...


D:\Installs\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


D:\Installs\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
D:\Installs\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Iteration: [160010/200000]
	D/loss_real = [-2.3522], D/loss_fake = [1.5287], D/loss_cls =  [0.1435], D/loss_gp = [0.0026]
	G/loss_fake = [-1.4656], G/loss_rec = [0.0716], G/loss_cls = [0.0714]
Iteration: [160020/200000]
	D/loss_real = [-2.6510], D/loss_fake = [2.0863], D/loss_cls =  [0.0800], D/loss_gp = [0.0042]
	G/loss_fake = [-2.1299], G/loss_rec = [0.0800], G/loss_cls = [0.3920]
Iteration: [160030/200000]
	D/loss_real = [-3.5695], D/loss_fake = [2.8483], D/loss_cls =  [0.1841], D/loss_gp = [0.0007]
	G/loss_fake = [-2.7953], G/loss_rec = [0.0992], G/loss_cls = [0.0939]
Iteration: [160040/200000]
	D/loss_real = [-3.7139], D/loss_fake = [2.4228], D/loss_cls =  [0.1816], D/loss_gp = [0.0119]
	G/loss_fake = [-2.1784], G/loss_rec = [0.1041], G/loss_cls = [0.6920]
Iteration: [160050/200000]
	D/loss_real = [-3.4477], D/loss_fake = [2.7561], D/loss_cls =  [0.0902], D/loss_gp = [0.0041]
	G/loss_fake = [-2.3771], G/loss_rec = [0.0968], G/loss_cls = [0.2767]
Iteration: [160060/200000]
	D/loss_

Iteration: [160440/200000]
	D/loss_real = [-2.4845], D/loss_fake = [1.8267], D/loss_cls =  [0.1300], D/loss_gp = [0.0001]
	G/loss_fake = [-1.5340], G/loss_rec = [0.0904], G/loss_cls = [0.1026]
Iteration: [160450/200000]
	D/loss_real = [-2.2785], D/loss_fake = [1.4460], D/loss_cls =  [0.0989], D/loss_gp = [0.0043]
	G/loss_fake = [-1.5354], G/loss_rec = [0.0961], G/loss_cls = [0.2071]
Iteration: [160460/200000]
	D/loss_real = [-2.5171], D/loss_fake = [1.9099], D/loss_cls =  [0.2025], D/loss_gp = [0.0057]
	G/loss_fake = [-1.7434], G/loss_rec = [0.0970], G/loss_cls = [0.5070]
Iteration: [160470/200000]
	D/loss_real = [-2.0922], D/loss_fake = [1.4746], D/loss_cls =  [0.1353], D/loss_gp = [0.0021]
	G/loss_fake = [-1.5127], G/loss_rec = [0.0936], G/loss_cls = [0.0803]
Iteration: [160480/200000]
	D/loss_real = [-2.6592], D/loss_fake = [1.9496], D/loss_cls =  [0.1068], D/loss_gp = [0.0039]
	G/loss_fake = [-1.9332], G/loss_rec = [0.0790], G/loss_cls = [0.0368]
Iteration: [160490/200000]
	D/loss_

...

Iteration: [178930/200000]
	D/loss_real = [-3.1337], D/loss_fake = [2.5251], D/loss_cls =  [0.1443], D/loss_gp = [0.0060]
	G/loss_fake = [-2.4717], G/loss_rec = [0.0761], G/loss_cls = [0.1352]
Iteration: [178940/200000]
	D/loss_real = [-3.1538], D/loss_fake = [2.5773], D/loss_cls =  [0.0981], D/loss_gp = [0.0056]
	G/loss_fake = [-2.5475], G/loss_rec = [0.0869], G/loss_cls = [0.2997]
Iteration: [178950/200000]
	D/loss_real = [-2.2437], D/loss_fake = [1.5222], D/loss_cls =  [0.1918], D/loss_gp = [0.0023]
	G/loss_fake = [-1.4935], G/loss_rec = [0.0802], G/loss_cls = [0.1279]
Iteration: [178960/200000]
	D/loss_real = [-2.5290], D/loss_fake = [1.8236], D/loss_cls =  [0.0566], D/loss_gp = [0.0048]
	G/loss_fake = [-1.8282], G/loss_rec = [0.0818], G/loss_cls = [0.0341]
Iteration: [178970/200000]
	D/loss_real = [-2.4923], D/loss_fake = [1.4962], D/loss_cls =  [0.1327], D/loss_gp = [0.0008]
	G/loss_fake = [-1.4483], G/loss_rec = [0.0738], G/loss_cls = [0.3289]
Iteration: [178980/200000]
	D/loss_

Iteration: [179360/200000]
	D/loss_real = [-2.5888], D/loss_fake = [1.8482], D/loss_cls =  [0.1854], D/loss_gp = [0.0037]
	G/loss_fake = [-1.8519], G/loss_rec = [0.0791], G/loss_cls = [0.1808]
Iteration: [179370/200000]
	D/loss_real = [-2.6045], D/loss_fake = [1.9598], D/loss_cls =  [0.2929], D/loss_gp = [0.0078]
	G/loss_fake = [-1.9372], G/loss_rec = [0.0619], G/loss_cls = [0.0694]
Iteration: [179380/200000]
	D/loss_real = [-2.4395], D/loss_fake = [1.6662], D/loss_cls =  [0.2909], D/loss_gp = [0.0006]
	G/loss_fake = [-1.6630], G/loss_rec = [0.0836], G/loss_cls = [0.5358]
Iteration: [179390/200000]
	D/loss_real = [-2.3171], D/loss_fake = [1.7741], D/loss_cls =  [0.2601], D/loss_gp = [0.0011]
	G/loss_fake = [-1.7583], G/loss_rec = [0.0812], G/loss_cls = [0.3040]
Iteration: [179400/200000]
	D/loss_real = [-2.6727], D/loss_fake = [2.0495], D/loss_cls =  [0.0862], D/loss_gp = [0.0004]
	G/loss_fake = [-2.0633], G/loss_rec = [0.0849], G/loss_cls = [0.0646]
Iteration: [179410/200000]
	D/loss_

Iteration: [179790/200000]
	D/loss_real = [-2.4473], D/loss_fake = [1.3839], D/loss_cls =  [0.0633], D/loss_gp = [0.0112]
	G/loss_fake = [-1.3600], G/loss_rec = [0.0933], G/loss_cls = [0.4530]
Iteration: [179800/200000]
	D/loss_real = [-2.3009], D/loss_fake = [1.6611], D/loss_cls =  [0.0540], D/loss_gp = [0.0044]
	G/loss_fake = [-1.5946], G/loss_rec = [0.0808], G/loss_cls = [0.3013]
Iteration: [179810/200000]
	D/loss_real = [-1.5649], D/loss_fake = [0.5612], D/loss_cls =  [0.2143], D/loss_gp = [0.0052]
	G/loss_fake = [-0.5206], G/loss_rec = [0.0854], G/loss_cls = [0.9836]
Iteration: [179820/200000]
	D/loss_real = [-2.6853], D/loss_fake = [1.2888], D/loss_cls =  [0.6386], D/loss_gp = [0.0052]
	G/loss_fake = [-1.2922], G/loss_rec = [0.0845], G/loss_cls = [0.1576]
Iteration: [179830/200000]
	D/loss_real = [-2.4565], D/loss_fake = [1.5402], D/loss_cls =  [0.2822], D/loss_gp = [0.0015]
	G/loss_fake = [-1.4277], G/loss_rec = [0.1104], G/loss_cls = [0.4840]
Iteration: [179840/200000]
	D/loss_

Iteration: [180220/200000]
	D/loss_real = [-2.7859], D/loss_fake = [2.4200], D/loss_cls =  [0.0852], D/loss_gp = [0.0010]
	G/loss_fake = [-2.2810], G/loss_rec = [0.0868], G/loss_cls = [0.0640]
Iteration: [180230/200000]
	D/loss_real = [-2.6072], D/loss_fake = [1.7359], D/loss_cls =  [0.3334], D/loss_gp = [0.0012]
	G/loss_fake = [-1.7613], G/loss_rec = [0.0912], G/loss_cls = [0.2526]
Iteration: [180240/200000]
	D/loss_real = [-2.6262], D/loss_fake = [1.9998], D/loss_cls =  [0.2607], D/loss_gp = [0.0045]
	G/loss_fake = [-1.8373], G/loss_rec = [0.0979], G/loss_cls = [0.1420]
Iteration: [180250/200000]
	D/loss_real = [-2.5505], D/loss_fake = [1.7932], D/loss_cls =  [0.1904], D/loss_gp = [0.0049]
	G/loss_fake = [-1.7884], G/loss_rec = [0.0746], G/loss_cls = [0.1819]
Iteration: [180260/200000]
	D/loss_real = [-2.2536], D/loss_fake = [1.3723], D/loss_cls =  [0.1989], D/loss_gp = [0.0026]
	G/loss_fake = [-1.3223], G/loss_rec = [0.0624], G/loss_cls = [0.2078]
Iteration: [180270/200000]
	D/loss_

Iteration: [180650/200000]
	D/loss_real = [-2.6749], D/loss_fake = [1.9328], D/loss_cls =  [0.1005], D/loss_gp = [0.0038]
	G/loss_fake = [-1.8386], G/loss_rec = [0.0748], G/loss_cls = [0.3947]
Iteration: [180660/200000]
	D/loss_real = [-2.2288], D/loss_fake = [1.1333], D/loss_cls =  [0.2918], D/loss_gp = [0.0045]
	G/loss_fake = [-1.0229], G/loss_rec = [0.0901], G/loss_cls = [0.2358]
Iteration: [180670/200000]
	D/loss_real = [-2.7011], D/loss_fake = [1.6304], D/loss_cls =  [0.0644], D/loss_gp = [0.0095]
	G/loss_fake = [-1.4852], G/loss_rec = [0.0940], G/loss_cls = [0.0262]
Iteration: [180680/200000]
	D/loss_real = [-2.4357], D/loss_fake = [1.4983], D/loss_cls =  [0.0660], D/loss_gp = [0.0024]
	G/loss_fake = [-1.4240], G/loss_rec = [0.0912], G/loss_cls = [0.1253]
Iteration: [180690/200000]
	D/loss_real = [-2.3935], D/loss_fake = [2.1414], D/loss_cls =  [0.1026], D/loss_gp = [0.0048]
	G/loss_fake = [-1.9842], G/loss_rec = [0.0958], G/loss_cls = [0.1661]
Iteration: [180700/200000]
	D/loss_

In [0]:
"""
This function is used to just test the generator model and generate iages from test data
"""
def test():
  mode='test'
  Image_data_class = ImageData(data_dir, selected_attributes)
  Image_data_class.preprocess()
  G_weights_dir = os.path.join(model_save_dir, 'G_weights.hdf5') 
  if not os.path.isfile(G_weights_dir):
    print("Don't find weight's generator model")
  else:
    G.load_weights(G_weights_dir)
  data_iter = get_loader(Image_data_class.test_dataset, Image_data_class.test_dataset_label, Image_data_class.test_dataset_fix_label, 
                          image_size, batch_size, mode)
  n_batches = int(sample_step / batch_size)
  # we are getting n_batches images from test dataset, getting the generator to generate the images and saving it
  for i in range(n_batches):
      imgs, orig_labels, target_labels, fix_labels, names = next(data_iter)
      #print("image length ",len(imgs))
      for j in range(batch_size):
          #print("fixed_labels", fix_labels[j])
          preds = G.predict([np.repeat(np.expand_dims(imgs[j], axis = 0), len(selected_attributes), axis = 0), fix_labels[j]])
          for k in range(len(selected_attributes)):  
              Image.fromarray((preds[k]*127.5 + 127.5).astype(np.uint8)).save(os.path.join(result_dir, names[j].split(os.path.sep)[-1].split('.')[0] + f'_{k + 1}.png')) 


In [0]:
test()

In [0]:
"""
Official Paper
https://arxiv.org/abs/1711.09020

Official Implementation in PyTorch
https://github.com/yunjey/stargan

Understanding StarGan
https://towardsdatascience.com/stargan-image-to-image-translation-44d4230fbb48
https://medium.com/@ashwinbhat2906/deep-learning-notes-stargan-f5506c2ce833

Residual Bottleneck
https://ai.stackexchange.com/questions/4864/what-is-the-concept-of-tensorflow-bottlenecks
https://towardsdatascience.com/understanding-residual-networks-9add4b664b03

Additional Links
https://github.com/hoangthang1607/StarGAN-Keras
https://keras.io/layers/about-keras-layers/
https://www.learnpython.org/en/Partial_functions






"""